In [3]:
from azure.cosmos import CosmosClient, exceptions, PartitionKey
import os
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
# Set the Cosmos DB endpoint, key and database name in the .env file. The key and endpoint can be found in the resource created in the portal.
COSMOS_ENDPOINT = os.environ["COSMOS_ENDPOINT"]
COSMOS_KEY =  os.environ["COSMOS_KEY"]
client = CosmosClient(COSMOS_ENDPOINT, credential=COSMOS_KEY)
DATABASE_NAME = 'contoso-outdoor'
CONTAINER_NAME = 'customers'

ServiceRequestError: No connection adapters were found for '<https://cosmos-contoso37283238.documents.azure.com:443/>'

In [ ]:
# Create the database if it doesnt already exist
client = CosmosClient(COSMOS_ENDPOINT, credential=COSMOS_KEY)
try:
    database = client.create_database(DATABASE_NAME)
except exceptions.CosmosResourceExistsError:
    database = client.get_database_client(DATABASE_NAME)

print(database)

In [ ]:
# Create the container if it doesnt already exist
try:
    container = database.create_container(id=CONTAINER_NAME, partition_key=PartitionKey(path="/id"))
except exceptions.CosmosResourceExistsError:
    container = database.get_container_client(CONTAINER_NAME)
except exceptions.CosmosHttpResponseError:
    raise
print(container)

In [ ]:
# Loop through each json file in data/customer_info and insert into container
import os
import json
import glob
path = '.'
for filename in glob.glob(os.path.join(path, '*.json')):
    with open(filename) as file:
        customer = json.load(file)
        container.upsert_item(customer)
        print('Upserted item with id {0}'.format(customer['id']))

In [ ]:
# Get items from container to validate they were inserted
print('Get all items in container')
items = list(container.read_all_items(max_item_count=10))
print(items)
